In [1]:
import numpy as np
from time import time

# Chapter 2<br>Systems of Equations

## 2.5 Iterative Methods

In [2]:
def Jacobi(A, b, iter_n, initial_guess=0):
    n = len(A)
    
    D = np.diag(A)
    R = A - np.diag(D)
    x_i = initial_guess * np.ones(n)
    
    for _ in range(iter_n):
        x_i = (b - R.dot(x_i)) / D
    
    return x_i

In [3]:
def Gauss_Seidel(A, b, iter_n, initial_guess=0):
    n = len(A)
    
    D = np.diag(A)
    L = np.tril(A) - np.diag(D)
    U = np.triu(A) - np.diag(D)
    
    x_i = initial_guess * np.ones(n)
    x_ii = x_i.copy()
    
    for _ in range(iter_n):
        for k in range(n):
            x_ii[k] = (b[k] - U[k].dot(x_i) - L[k].dot(x_ii)) / D[k]
        x_i = x_ii.copy()
        
    return x_i

In [4]:
def SOR(A, b, w, iter_n, initial_guess=0):
    n = len(A)
    
    D = np.diag(A)
    L = np.tril(A) - np.diag(D)
    U = np.triu(A) - np.diag(D)
    
    x_i = initial_guess * np.ones(n)
    
    for _ in range(iter_n):
        x_i = np.linalg.inv(w*L + np.diag(D)).dot((1 - w)*np.diag(D).dot(x_i) - w*U.dot(x_i)) + \
        w*np.linalg.inv(np.diag(D) + w*L).dot(b)
        
    return x_i

### Q. 1

In [5]:
def A_ij(n):
    A = np.zeros((n, n))
    for i in range(n):
        A[i, i] = 3
        if i < n-1:
            A[i, i+1] = -1
            A[i+1, i] = -1

    return A

In [6]:
# n = 100
n = 100

A = A_ij(n)
b = np.ones(n)
b[0], b[-1] = 2, 2
x_true = np.ones(n)

In [7]:
j = 0
guess = np.zeros(n)

while True:
    guess = Jacobi(A, b, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-7:
        break

backward_error = abs(A.dot(guess) - b).max()

In [8]:
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

Number of steps needed: 36
Backward error: 0.0000004579


In [9]:
# n = 100000
n = 100000

# Due to Memory Error, directly calculate the equation.
# A = A_ij(n)
b = np.ones(n)
b[0], b[-1] = 2, 2
x_true = np.ones(n)

In [10]:
j = 0
guess = np.zeros(n)

while True:
#     guess = Jacobi(A, b, 1, guess)
    guess = (b + np.concatenate((guess[1:], [0])) + np.concatenate(([0], guess[:-1]))) / 3
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-7:
        break

backward_error = abs((3*guess - np.concatenate((guess[1:], [0])) - np.concatenate(([0], guess[:-1]))) - b).max()

In [11]:
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

Number of steps needed: 36
Backward error: 0.0000004579


### Q. 2

In [12]:
n = 100
A = np.zeros((n, n))
for i in range(n):
    A[i, i] = 2
    if i < n-1:
        A[i, i+1] = 1
        A[i+1, i] = 1
        
b = np.zeros(n)
b[0], b[-1] = 1, -1
x_true = np.ones(n)
x_true[1::2] -= 2

In [13]:
j = 0
guess = np.zeros(n)

while True:
    guess = Jacobi(A, b, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-4:
        break
    
backward_error = abs(A.dot(guess) - b).max()

In [14]:
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

Number of steps needed: 16209
Backward error: 0.0000004836


### Q. 3

In [15]:
A = np.zeros((6, 6))
for i in range(6):
    A[i, 5-i] = 0.5
    A[i, i] = 3
    if i > 0:
        A[i, i-1] = -1
        A[i-1, i] = -1
b = np.array([2.5, 1.5, 1, 1, 1.5, 2.5])
x_true = np.ones(6)

In [16]:
guess = np.zeros(6)
guess = Gauss_Seidel(A, b, 6, guess)

In [17]:
print("Gauss-Seidel:", np.round(guess, 4))

Gauss-Seidel: [0.995  0.9946 0.9969 0.9996 1.0016 1.0013]


### Q. 4

In [18]:
A = np.zeros((6, 6))
for i in range(6):
    A[i, 5-i] = 0.5
    A[i, i] = 3
    if i > 0:
        A[i, i-1] = -1
        A[i-1, i] = -1
b = np.array([2.5, 1.5, 1, 1, 1.5, 2.5])
x_true = np.ones(6)

In [19]:
w = 1.1
guess = np.zeros(6)
guess = SOR(A, b, w, 6, guess)

In [20]:
print("SOR:", np.round(guess, 4))

SOR: [0.9989 0.9993 1.0004 1.0009 1.0009 1.0004]


### Q. 5

In [21]:
n = 100

A = np.zeros((n, n))
for i in range(n):
    A[i, i] = 3
    if i < n-1:
        A[i, i+1] = -1
        A[i+1, i] = -1

b = np.ones(n)
b[0], b[-1] = 2, 2
x_true = np.ones(n)

In [22]:
# (a)
j = 0
guess = np.zeros(n)

while True:
    guess = Gauss_Seidel(A, b, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-7:
        break

backward_error = abs(A.dot(guess) - b).max()

In [23]:
print("Gauss-Seidel Method")
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

Gauss-Seidel Method
Number of steps needed: 21
Backward error: 0.0000004779


In [24]:
# (b)
j = 0
w = 1.2
guess = np.zeros(n)

while True:
    guess = SOR(A, b, w, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-7:
        break

backward_error = abs(A.dot(guess) - b).max()

In [25]:
print("SOR Method")
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

SOR Method
Number of steps needed: 16
Backward error: 0.0000015516


### Q. 6

In [26]:
n = 100
A = np.zeros((n, n))
for i in range(n):
    A[i, i] = 2
    if i < n-1:
        A[i, i+1] = 1
        A[i+1, i] = 1
        
b = np.zeros(n)
b[0], b[-1] = 1, -1
x_true = np.ones(n)
x_true[1::2] -= 2

In [27]:
# (a)
j = 0
guess = np.zeros(n)

while True:
    guess = Gauss_Seidel(A, b, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-4:
        break
    
backward_error = abs(A.dot(guess) - b).max()

In [28]:
print("Gauss-Seidel Method")
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

Gauss-Seidel Method
Number of steps needed: 8106
Backward error: 0.0000004836


In [29]:
# (b)
j = 0
w = 1.5
guess = np.zeros(n)

while True:
    guess = SOR(A, b, w, 1, guess)
    j += 1    
    forward_error = abs(guess - x_true).max()
    if forward_error < 5e-4:
        break
    
backward_error = abs(A.dot(guess) - b).max()

In [30]:
print("SOR Method")
print("Number of steps needed: %d" % j)
print("Backward error: %.10f" % backward_error)

SOR Method
Number of steps needed: 2699
Backward error: 0.0000004858


### Q. 7

In [31]:
for n in [100, 200, 500, 1000, 2000, 5000, 10000]:
    A = np.zeros((n, n))
    for i in range(n):
        A[i, n-1-i] = 0.5
        A[i, i] = 3
        if i > 0:
            A[i, i-1] = -1
            A[i-1, i] = -1

    b = 1.5 * np.ones(n)
    b[0], b[-1] = 2.5, 2.5
    b[n//2], b[n//2 - 1] = 1, 1
    x_true = np.ones(n)

    guess = np.zeros(n)

    tic = time()
    while True:
        guess = Gauss_Seidel(A, b, 1, guess)
        toc = time()
        if toc - tic > 1:
            break
    forward_error = abs(guess - x_true).max()
    
    print("Size: %5d / Forward Error: %f" % (n, forward_error))

Size:   100 / Forward Error: 0.000000
Size:   200 / Forward Error: 0.000000
Size:   500 / Forward Error: 0.000000
Size:  1000 / Forward Error: 0.000002
Size:  2000 / Forward Error: 0.013012
Size:  5000 / Forward Error: 0.216520
Size: 10000 / Forward Error: 0.472222
